## Lecture 19 - Interpolation & Integration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d

### 1D interpolation

In [ ]:
x = np.linspace(0, 10, num=11, endpoint=True)
y = np.cos(-x**2/9.0)
f1 = interp1d(x, y, kind='nearest')
f2 = interp1d(x, y, kind='previous')
f3 = interp1d(x, y, kind='next')

In [ ]:
xnew = np.linspace(0, 10, num=1001, endpoint=True)
plt.plot(x, y, 'o')
plt.plot(xnew, f1(xnew), '-', xnew, f2(xnew), '--', xnew, f3(xnew), ':')
plt.legend(['data', 'nearest', 'previous', 'next'], loc='best')
plt.show()

In [ ]:
x = np.linspace(0, 10, num=11)
y = np.cos(-x**2 / 9.0)

plt.plot(x, y, 'o')

### Piecewise linear interpolation (Connect the dots)

In [ ]:
xnew = np.linspace(0, 10, num=1001)
ynew = np.interp(xnew, x, y)

In [ ]:
plt.plot(x, y, 'o')
plt.plot(xnew, ynew, '-')


### Cubic splines

In [ ]:
from scipy import interpolate

In [ ]:
spline = interpolate.CubicSpline(x, y)

This creates a **function** that will take a value of $x$ as input and outputs the interpolated value of $y$ at $x$.

In [ ]:
print(spline(2))

In [ ]:
plt.plot(x, y, 'o')
plt.plot(xnew, spline(xnew), '-')
plt.show()

This gives you a *smoother* interpolation.

### 2D interpolation

In [ ]:
x_edges, y_edges = np.mgrid[-1:1:21j, -1:1:21j]
x = x_edges[:-1, :-1] + np.diff(x_edges[:2, 0])[0] / 2.
y = y_edges[:-1, :-1] + np.diff(y_edges[0, :2])[0] / 2.
z = (x+y) * np.exp(-6.0*(x*x+y*y))

In [ ]:
plt.figure()
lims = dict(cmap='RdBu_r', vmin=-0.25, vmax=0.25)
plt.pcolormesh(x_edges, y_edges, z, shading='flat', **lims)
plt.colorbar()
plt.title("Sparsely sampled function.")
plt.show()

In [ ]:
xnew_edges, ynew_edges = np.mgrid[-1:1:71j, -1:1:71j]
xnew = xnew_edges[:-1, :-1] + np.diff(xnew_edges[:2, 0])[0] / 2.
ynew = ynew_edges[:-1, :-1] + np.diff(ynew_edges[0, :2])[0] / 2.
tck = interpolate.bisplrep(x, y, z, s=0)
znew = interpolate.bisplev(xnew[:,0], ynew[0,:], tck)

In [ ]:
plt.figure()
plt.pcolormesh(xnew_edges, ynew_edges, znew, shading='flat', **lims)
plt.colorbar()
plt.title("Interpolated function.")
plt.show()

### Numerical integration 

In [ ]:
def y(x):
    return np.cos(-x**2/9.0) + 2.5

true_integral = 26.4333

In [ ]:
# Mid-point rule

x = np.linspace(0, 10, 1000)

fig, axs = plt.subplots(1, 3, figsize=(10, 3.5), dpi=120, sharex=True, sharey=True, gridspec_kw={'wspace':0.05})

nsteps_list = [5, 10, 25]



for i, n in enumerate(nsteps_list):
    axs[i].plot(x, y(x), '-')
    axs[i].set_xmargin(0)

    x_int = np.linspace(0, 10, n+1)
    y_int = np.cos(-x_int**2 / 9.0) + 2.5

    dx = x_int[1:] - x_int[:-1]
    x_mid = (x_int[1:] + x_int[:-1]) / 2
    y_mid = (y_int[1:] + y_int[:-1]) / 2

    midpoint_int = np.sum(y_mid * dx)

    axs[i].bar(x_mid, y_mid, width=dx, align='center', edgecolor='black', alpha=0.5, label=r'$\int y(x)\, \mathrm{d}x \approx$' f'{midpoint_int:.4f}')
    axs[i].set_title(f'Midpoint integration with {n} steps', size=10)
    axs[i].legend(loc='lower left')
fig.suptitle('$\int y(x)\, \mathrm{d}x = 26.4333$\n -------------------------------------------', size=12, y=1.1)

In [ ]:
# Trapezoid rule


fig, axs = plt.subplots(1, 3, figsize=(10, 3.5), dpi=120, sharex=True, sharey=True, gridspec_kw={'wspace':0.05})


nsteps_list = [5, 10, 25]

for i, n in enumerate(nsteps_list):
    axs[i].plot(x, y(x), '-', c='k', lw=1, zorder=3)
    axs[i].set_xmargin(0)

    x_int = np.linspace(0, 10, n+1)
    y_int = np.cos(-x_int**2 / 9.0) + 2.5

    dx = x_int[1:] - x_int[:-1]

    trap_int = 0

    for j in range(1, len(x_int)):
        x0 = x_int[j-1]
        x1 = x_int[j]

        trap_int += (x1 - x0) * (y(x0) + y(x1)) / 2

    axs[i].plot(x_int, y_int, lw=1, color='red')
    axs[i].fill_between(x_int, y_int, alpha=0.5, color='C0', label=r'$\int y(x)\, \mathrm{d}x \approx$' f'{trap_int:.4f}')
    axs[i].vlines(x_int, 0, y_int, color='red', lw=0.8)

    axs[i].set_title(f'Trapezoid rule with {n} steps', size=10)
    axs[i].legend(loc='lower left')
axs[0].set_ylim(0, 3.9)

fig.suptitle('$\int y(x)\, \mathrm{d}x = 26.4333$\n -------------------------------------------', size=12, y=1.1)


In [ ]:
# # Simpsons rule

# fig, axs = plt.subplots(1, 3, figsize=(10, 3.5), dpi=120, sharex=True, sharey=True, gridspec_kw={'wspace':0.05})

# nsteps_list = [6, 10, 26]

print('True integral:', true_integral)

for i, n in enumerate(nsteps_list):
    # axs[i].plot(x, y(x), '-', c='k', lw=1, zorder=3)
    # axs[i].set_xmargin(0)

    x_int = np.linspace(0, 10, n+1)
    y_int = np.cos(-x_int**2 / 9.0) + 2.5

    simp_int = 0


    for j in range(2, len(x_int), 2):
        x0 = x_int[j-2]
        x1 = x_int[j-1]
        x2 = x_int[j]

        dx = x1 - x0

        simp_int += dx * (y(x0) + 4 * y(x1) + y(x2)) / 3

    print(f'Simpsons rule with {n} steps: {simp_int:.4f}')

#         a =  (y(x0) - 2 * y(x1) + y(x2))/(2 * dx**2)
#         b = (dx * y(x0) + dx * y(x2) - 2 * x1 * y(x1) +  4 * x1 * y(x1) - 2 * x1 * y(x2))/(2 * dx**2)
#         c = y(x1) + (dx * x1 * y(x0)- dx * x2 * y(x2) + x1**2 * y(x0) - 2 * x1**2 * y(x1)+ x1**2 * y(x2))/(2 * dx**2)

#         x_quad = np.linspace(x0, x2, 100)
#         y_quad = a * (x_quad - dx)**2 + b * (x_quad - dx) + c

#         axs[i].plot(x_quad, y_quad, lw=1, color='red')
#         if j < len(x_int) - 1:
#             axs[i].fill_between(x_quad, y_quad, alpha=0.5, color='C0')
#         else:
#             axs[i].fill_between(x_quad, y_quad, alpha=0.5, color='C0', label=r'$\int y(x)\, \mathrm{d}x \approx$' f'{simp_int:.4f}')
#     axs[i].vlines(x_int, 0, y_int, color='red', lw=0.8)

#     axs[i].set_title(f'Simpsons rule with {n} steps', size=10)
#     axs[i].legend(loc='lower left')
# axs[0].set_ylim(0, 3.9)

# fig.suptitle('$\int y(x)\, \mathrm{d}x = 26.4333$\n -------------------------------------------', size=12, y=1.1)

In [ ]:
from scipy import integrate